In [0]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [0]:
im=[]
f=[]
m=[]
blacklist = ['[document]','noscript','header','html','meta','head', 'input','script']

In [12]:
for pagenum in range(1,3):
    html = urlopen('https://www.imdb.com/list/ls068010962/?sort=list_order,asc&mode=detail&page='+str(pagenum))
    bs = BeautifulSoup(html, 'html.parser')
    images = bs.find_all('img', {'src':re.compile('.jpg')}) # extract all image links from page
    for image in images: 
        im.append(image['src']+'\n') # append image
        f.append(image['alt']) # append celebrity name
    for links in bs.find_all('a'):
        link=links.get('href')
        try:
            if(link[0:6]=='/name/' and link[-1]=='/'):
                m.append('https://www.imdb.com'+link+'bio?ref_=nm_ov_bio_sm') # append Bio page of celebrity
        except Exception:
            pass
    print(len(im),len(f),len(m))

98 98 100
197 197 200


In [0]:
unwanted_num = {m[23],m[54],m[188]} # there are 3 celebrities with no image. Remove them.  
m = [ele for ele in m if ele not in unwanted_num]

In [14]:
print(len(im),len(f),len(m))

197 197 197


In [0]:
trademark=[]
height=[]
sub_str = [ 'Overview','Mini Bio','Trade Mark','Trivia']
occurrence = 2
for url in m:
    occ=[]
    data=urlopen(url)
    soup=BeautifulSoup(data,'html.parser')
    ############### STORE TEXT FROM NOT BLACKLISTED TAGS ###########
    text = soup.find_all(text=True)
    output = ''
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    ################# STORE POSITION OF HEADERS IN URL ##############
    for sub in sub_str:
        val = -1
        for i in range(0, occurrence): 
            val = output.find(sub, val + 1) 
        occ.append(val)
    ################## STORE TRAITS OF PERSONALITY ###################
    if(occ[2]==-1):
        trademark.append([])
    else:
        if(occ[3]!=-1):
            k=output[occ[2]:occ[3]]
            x=k.split('\n \n')
            for i in range(len(x)):
                x[i]=x[i].strip()
            if(len(x[1:-1])>2):
                trademark.append(x[1:-1][:2])
            else:
                trademark.append(x[1:-1])
        else:
            k=output[occ[2]:]
            trademark.append(k.split('\n \n')[1].strip())
    ####################STORE HEIGHT OF PERSONALITY ##################
    if(occ[0]==-1):
        height.append('')
    else:
        if(occ[1]==-1):
            height.append('')
        else:
            r=output[occ[0]:occ[1]]
            x=r.split('\n')
            for i in range(len(x)):
                x[i]=x[i].strip()
            without_empty_strings = []
            for string in x:
                if (string != ""):
                    without_empty_strings.append(string)
            count=0
            for i in without_empty_strings[-1]:
                if(i=='.'):
                    break
                else:
                    count=count+1
            a=without_empty_strings[-1][count-1:count+2]
            if(a.replace('.','',1).isdigit() and a.count('.') < 2):
                height.append(a)
            else:
                height.append('')

In [0]:
############## APPEND HEIGHT TO TRAIT ###########################
for i in range(len(height)):
    if(type(trademark[i])==list and height[i]!=''):
        trademark[i].append('height is '+str(height[i])+' m')
############# MAKE EMPTY LIST as NaN ###########################
for i in range(len(trademark)):
    if(trademark[i]==[]):
        trademark[i]=np.nan

In [18]:
df=pd.DataFrame(zip(f,im,m,trademark),columns=['Celebrity','Image','Bio','Trait'])
df.head()

,Celebrity,Image,Bio,Trait
0,Aamir Khan,https://m.media-amazon.com/images/M/MV5BMjAwMj...,https://www.imdb.com/name/nm0451148/bio?ref_=n...,"[Short, yet muscular stature, Frequently colla..."
1,Akshay Kumar,https://m.media-amazon.com/images/M/MV5BODI4ND...,https://www.imdb.com/name/nm0474774/bio?ref_=n...,"[Performs his own stunta, Heavy, yet raspy voi..."
2,Ajay Devgn,https://m.media-amazon.com/images/M/MV5BMTY0MD...,https://www.imdb.com/name/nm0222426/bio?ref_=n...,"[Intense Eyes, Chocolatey Eyes, height is 1.7 m]"
3,Amjad Khan,https://m.media-amazon.com/images/M/MV5BOWUzNj...,https://www.imdb.com/name/nm0451166/bio?ref_=n...,[height is 1.7 m]
4,Amitabh Bachchan,https://m.media-amazon.com/images/M/MV5BNTk1OT...,https://www.imdb.com/name/nm0000821/bio?ref_=n...,"[His baritone voice, height is 1.8 m]"
